# Check GPU

In [1]:
!nvidia-smi

Wed Jun 30 14:55:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Step 1:  
Mount Google Drive to the Colab Notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Step 2:
Make a Directory **yolov4** in Google Drive and another Directory **training** inside **yolov4**

In [ ]:
#!pwd
!cd drive/MyDrive && mkdir yolov4 && mkdir yolov4/training

/bin/bash: line 0: cd: drive/MyDrive: No such file or directory


# Step 3:
Navigate to **yolov4** Directory

In [ ]:
%cd drive/MyDrive/yolov4

/content/drive/MyDrive/yolov4


# Step 4:
Clone the Darknet git repository from AlexeyAB into **yolov4** Directory on Drive

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

# Step 5:
Upload following files to the **yolov4** Directory on Drive :


*   Labelled Dataset
*   Custom cfg file
*   obj.data and obj.names files
*   train.txt and test.txt files

# Step 5(a) :
Copy all the images and annotation txt files to a folder called **obj** and compress the directory. Upload **obj.zip** to **yolov4** directory on drive.

# Step 5(b) :
Download the *yolov4-custom.cfg* file from **darknet/cfg** directory.
Make the following changes :


*   batch = 64
*   subdivisions = 16
*   max_batches = num_classes*2000 (but not less than 6000)
*   steps = 4800, 5400 (80% of max_batches, 90% of max_batches)
*   filters = (num_classes + 5)x3 in 3 **[convolutional]** layers before each **[yolo]** layers.
*   classes = 2 (num_classes) in each of 3 **[yolo]** layers.

Then upload it to **yolov4** directory.


# Step 5(c) :


*   Create obj.data, which contains following lines -


1.   classes = 2
2.   train = data/train.txt
3.   valid = data/test.txt
4.   names = data/obj.names
5.   backup = /content/drive/MyDrive/yolov4/training

*   Create obj.names, which contains following lines -


1.   ball
2.   player


Then, upload them to **darknet** directory.

# Step 5(d) :
Upload train.txt and test.txt to **yolov4** which were create during data pre-processing phase.

# Step 6:
Make changes in the makefile to enable OPENCV and GPU.

In [ ]:
%cd darknet/
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

/content/drive/MyDrive/yolov4/darknet


# Step 7:
Run make command to build darknet

In [ ]:
!make

# Step 8:
Copy all the files from the **yolov4** folder to the **darknet** directory

# Step 8(a):
Unzip the **obj.zip** dataset and its contents so that they are now in **/darknet/data/** folder

In [ ]:
!unzip /content/drive/MyDrive/yolov4/obj.zip -d data/

# Step 8(b):
Copy **yolov4-custom.cfg** file so that it is now in **/darknet/cfg/** folder

In [ ]:
%rm -rf cfg/
%mkdir cfg
!cp /content/drive/MyDrive/yolov4/yolov4-custom.cfg cfg

# Step 8(c):
Copy the **obj.names, obj.data, train.txt, test.txt** files so that they are now in **/darknet/data/** folder

In [ ]:
!cp /content/drive/MyDrive/yolov4/obj.names data
!cp /content/drive/MyDrive/yolov4/obj.data  data
!cp /content/drive/MyDrive/yolov4/train.txt data
!cp /content/drive/MyDrive/yolov4/test.txt  data

!ls data/

# Step 9:
Download the pre-trained YOLOv4 weights

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

# Step 10:
Train your custom detector

In [ ]:
!./darknet detector train data/obj.data cfg/yolov4-custom.cfg yolov4.conv.137 -dont_show -map

# Step 11:
To Resume your training

In [ ]:
!./darknet detector train data/obj.data cfg/yolov4-custom.cfg /content/drive/MyDrive/yolov4/training/yolov4-custom_last.weights -dont_show -map

# Step 12:
Check the training chart

In [ ]:
def imShow(path):import cv2
import matplotlib.pyplot as plt
%matplotlib inline
image = cv2.imread(path)
height, width = image.shape[:2]
resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)
fig = plt.gcf()
fig.set_size_inches(18, 10)
plt.axis(“off”)
plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
plt.show()

imShow('chart.png')

# Step 13:
Check mAP (mean average precision)
xxxx means iteration no.(1000, 2000 etc.)

In [ ]:
!./darknet detector map data/obj.data cfg/yolov4-custom.cfg /content/drive/MyDrive/yolov4/training/yolov4-custom_xxxx.weights -points 0

# Step 14:
Test your custom Object Detector

# Step 14(a):
Make changes to your custom config file to set it to test mode

In [ ]:
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4-custom.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-custom.cfg
%cd ..

# Step 14(b):
Run detector on an image

In [ ]:
!./darknet detector test data/obj.data cfg/yolov4-custom.cfg /content/drive/MyDrive/yolov4/training/yolov4-custom_best.weights /content/drive/MyDrive/test_image1.jpg -thresh 0.3
imShow('predictions.jpg')

# Step 14(c):
Run detector on a video

In [ ]:
!./darknet detector demo data/obj.data cfg/yolov4-custom.cfg /content/drive/MyDrive/yolov4/training/yolov4-custom_best.weights -dont_show /content/drive/MyDrive/test1.mp4 -thresh 0.5 -i 0 -out_filename /content/drive/MyDrive/results1.avi